In [ ]:
# !pip install pinecone-client

In [2]:
import pinecone

In [3]:
# Pinecone 클라이언트를 초기화합니다.
pinecone.init(      
	api_key='dec6a022-886e-487c-8c37-0a8e82114296',      
	environment='us-east1-gcp'      
)

In [22]:
if not pinecone.list_indexes():
    pinecone.create_index('sementic-test',
                          dimension=128,
                          metric='euclidean')

In [12]:
pinecone.describe_index('sementic-test')

IndexDescription(name='sementic-test', metric='euclidean', replicas=1, dimension=128.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

### insert One

In [13]:
import random

ids = ['a','b','c','d','e']
vecs = [[random.random() for _ in range(128)] for vec in range(5)]

In [15]:
index = pinecone.Index('sementic-test')

In [16]:
index.upsert(vectors=zip(ids,vecs))

{'upserted_count': 5}

### insert Batch

In [23]:
import itertools

vector_dim = 128
vector_count = 10000

# Example generator that generates many (id, vector) pairs
example_data_generator = map(
    lambda i:
        (f'id-{i}', [random.random() for _ in range(vector_dim)]),
        range(vector_count) 
)

In [25]:
def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size)) 

In [20]:
for chunk in chunks(example_data_generator):
    index.upsert(vectors=chunk)

## Insert Parallel

In [26]:
with pinecone.Index('sementic-test', pool_threads=30) as index:
    async_results = [
        index.upsert(vectors=ids_vectors_chunk, async_req=True)
        for ids_vectors_chunk in chunks(example_data_generator, batch_size=100)
    ]
    
    [async_result.get() for async_result in async_results]

## Fetch Vectors

In [28]:
index.fetch(ids=['id-0', 'id-1'])

In [31]:
# update vectors
index.upsert(vectors=[('id-0', [0.0] * 128)])

{'upserted_count': 1}

In [33]:
# Deleting Vectors
index.delete(ids=['id-1'])

index.delete(delete_all=True, namespace='example-namespace')

{}